* STEP 1 — Imports

In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score


data = {
    "age": [25,45,35,50,23,40,29,55,31,48,26,37],
    "income": [30000,120000,60000,150000,25000,90000,35000,200000,45000,130000,28000,75000],
    "loan_amount": [200000,500000,300000,800000,150000,400000,250000,900000,270000,600000,180000,350000],
    "credit_score": [650,720,680,750,640,700,660,780,670,740,645,690],
    "default": [0,0,0,0,1,0,1,0,1,0,1,0]
}

df = pd.DataFrame(data)
df.head()


,age,income,loan_amount,credit_score,default
0,25,30000,200000,650,0
1,45,120000,500000,720,0
2,35,60000,300000,680,0
3,50,150000,800000,750,0
4,23,25000,150000,640,1


* STEP 2 — Feature / Target Split

In [8]:
X = df.drop("default", axis=1)
y = df["default"]


* STEP 3 — Train / Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)


* STEP 4 — kNN WITHOUT Scaling (Wrong way)

In [10]:
knn_no_scaling = KNeighborsClassifier(
    n_neighbors=5,
    metric="euclidean"
)

knn_no_scaling.fit(X_train, y_train)

train_auc_ns = roc_auc_score(
    y_train,
    knn_no_scaling.predict_proba(X_train)[:,1]
)

test_auc_ns = roc_auc_score(
    y_test,
    knn_no_scaling.predict_proba(X_test)[:,1]
)

print("No Scaling - Train AUC:", train_auc_ns)
print("No Scaling - Test  AUC:", test_auc_ns)


No Scaling - Train AUC: 0.9
No Scaling - Test  AUC: 1.0


`What this shows`

Income & loan_amount dominate distance

Age & credit_score almost ignored

Distance bias present

* STEP 5: kNN WITH Scaling(Correct way)

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

knn_scaled = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

knn_scaled.fit(X_train, y_train)

train_auc_s = roc_auc_score(
    y_train, knn_scaled.predict_proba(X_train)[:, 1]
)
test_auc_s = roc_auc_score(
    y_test, knn_scaled.predict_proba(X_test)[:, 1]
)

print("\nWITH Scaling")
print("Train AUC:", train_auc_s)
print("Test  AUC:", test_auc_s)



WITH Scaling
Train AUC: 0.9
Test  AUC: 1.0


C:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^

`What this shows`

All features contribute fairly

Distance becomes meaningful

Generalization improves

* STEP 6: Bias–Variance via k

In [13]:
k_values = [1, 3, 5, 7]

for k in k_values:
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=k))
    ])
    
    model.fit(X_train, y_train)
    
    train_auc = roc_auc_score(
        y_train, model.predict_proba(X_train)[:, 1]
    )
    
    test_auc = roc_auc_score(
        y_test, model.predict_proba(X_test)[:, 1]
    )
    
    print(f"k={k} | Train AUC={train_auc:.3f} | Test AUC={test_auc:.3f}")


k=1 | Train AUC=1.000 | Test AUC=1.000
k=3 | Train AUC=0.900 | Test AUC=1.000
k=5 | Train AUC=0.900 | Test AUC=1.000
k=7 | Train AUC=0.700 | Test AUC=0.833


---

# 📌 kNN & Distance Bias — Cheat Sheet (Interview-Ready)

---

## 1️⃣ What is kNN?

* Distance-based supervised algorithm
* No training phase (lazy learner)
* Prediction = nearest `k` neighbors’ vote/average

---

## 2️⃣ Distance Bias

**Definition:**
When features with larger numeric scales dominate distance calculation.

**Key Rule:**

> **kNN without feature scaling is mathematically wrong.**

* Scaling required: `StandardScaler` / `MinMaxScaler`
* Trees don’t need scaling; kNN does

---

## 3️⃣ Curse of Dimensionality

* As feature count ↑, all points become similarly distant
* Nearest neighbor ≈ farthest neighbor
* Distance loses meaning → kNN collapses

**One-liner:**

> “High dimensionality causes distance concentration.”

---

## 4️⃣ Bias–Variance Control in kNN (via `k`)

| k value       | Behavior    | Risk                            |
| ------------- | ----------- | ------------------------------- |
| Small k (1,3) | Very local  | **High variance (overfitting)** |
| Medium k      | Balanced    | Best tradeoff                   |
| Large k       | Over-smooth | **High bias (underfitting)**    |

---

## 5️⃣ Why **k = 1** is Dangerous (IMPORTANT)

Even if:

```
Train AUC = 1.0
Test  AUC = 1.0
```

### ❌ Still NOT balanced

**Why?**

* Uses only **one data point**
* Extremely sensitive to noise
* **Stability is very low**
* A single point moving/changing can flip predictions

👉 High test score here is due to **small dataset luck**, not true generalization.

**Interview-ready line:**

> “k=1 has extremely low stability; a small perturbation or noisy point can completely change predictions, indicating very high variance.”

---

## 6️⃣ How to Identify “Best Balanced” Model

❌ Wrong logic:

> Highest test AUC

✅ Correct logic:

* Train score not perfect
* Train–test gap small
* Model behavior stable
* Less sensitive to small data changes

👉 For small datasets: **prefer k = 3 or 5 over k = 1**

---

## 7️⃣ Why Step-6 (varying k) is Mandatory in kNN

* kNN has **no regularization**
* No coefficients
* `k` is the **only complexity control knob**

👉 So:

> **Tuning and analyzing k is mandatory for every kNN model**

Yes — **this applies to all kNN use cases**.

---

## 8️⃣ Final One-Line Summary

> “kNN performance depends entirely on distance and k; small k leads to high variance and instability, while larger k increases bias. Metrics alone can be misleading on small datasets.”

---

## 9️⃣ Ultra-Short Memory Hooks

* kNN → distance-based → scaling mandatory
* Small k → memorization → high variance
* Large k → over-smoothing → high bias
* k = 1 + perfect score ≠ good model

---


Got it. Below is a **clear, structured:

* **kNN advantages**
* **kNN disadvantages & limitations**
* **Why kNN is fundamentally different from Linear & Logistic Regression**
* **Direct comparisons you can say in interviews**

No fluff. This is the version you revise before interviews.

---

# 📌 kNN (k-Nearest Neighbors): Advantages, Disadvantages & Comparison

---

## 1️⃣ What makes kNN different (Big Picture)

### kNN

* **Instance-based / distance-based algorithm**
* No explicit training phase
* Stores data and decides **at prediction time**

### Linear / Logistic Regression

* **Parametric models**
* Learn coefficients during training
* Use a fixed mathematical form

👉 This single difference explains almost all behavior differences.

---

## 2️⃣ Advantages of kNN

### ✅ Advantage 1: Simple and Intuitive

* Easy to understand and explain
* “Similar points should have similar labels”

Good for:

* Learning ML intuition
* Quick baselines

---

### ✅ Advantage 2: No Training Phase

* Training time ≈ zero
* Just stores data

Good for:

* Small datasets
* Rapid experimentation

---

### ✅ Advantage 3: Can Model Non-Linear Patterns

* Decision boundary can be very flexible
* No assumption of linearity

Why this matters:

* Logistic Regression has a **linear decision boundary**
* kNN can follow complex shapes

---

### ✅ Advantage 4: Assumption-Free

* No assumption about data distribution
* No assumption of linear relationship

---

## 3️⃣ Disadvantages of kNN (Critical)

### ❌ Disadvantage 1: Feature Scaling is Mandatory

* kNN uses distance
* Large-scale features dominate distance

Without scaling:

* Distance becomes meaningless
* Predictions are wrong

👉 In Logistic Regression: scaling is important
👉 In kNN: scaling is **mandatory**

---

### ❌ Disadvantage 2: Curse of Dimensionality

* As feature count increases:

  * All points become similarly distant
  * Nearest neighbor ≈ farthest neighbor

Result:

* kNN loses discriminative power
* Performance collapses

---

### ❌ Disadvantage 3: Slow Prediction

* Prediction requires computing distance to **all training points**
* Very slow for large datasets

👉 Logistic Regression prediction is fast.

---

### ❌ Disadvantage 4: High Memory Usage

* Entire training dataset must be stored
* Model size = data size

---

### ❌ Disadvantage 5: Noise Sensitivity

* Especially for small k (e.g., k=1)
* One noisy point can flip predictions

👉 High variance problem

---

## 4️⃣ Limitations of kNN

kNN performs poorly when:

* Dataset is large
* Feature space is high-dimensional
* Data is noisy
* Interpretability is required
* Low-latency prediction is needed

---

## 5️⃣ kNN vs Linear / Logistic Regression (Direct Comparison)

| Aspect                  | kNN            | Linear / Logistic Regression |
| ----------------------- | -------------- | ---------------------------- |
| Model type              | Instance-based | Parametric                   |
| Training                | No training    | Learns coefficients          |
| Prediction speed        | Slow           | Fast                         |
| Feature scaling         | Mandatory      | Important, not mandatory     |
| Handles high dimensions | Poorly         | Better                       |
| Interpretability        | None           | High                         |
| Bias–variance control   | Via k          | Via regularization           |
| Production usage        | Rare           | Very common                  |

---

## 6️⃣ Bias–Variance Comparison

### kNN

* Small k → **High variance** (overfitting)
* Large k → **High bias** (underfitting)

### Logistic Regression

* High bias by design (linear boundary)
* Variance controlled using **L1 / L2 regularization**
* Much more stable

---

## 7️⃣ Why kNN behaves differently from Logistic Regression

### kNN

* Local, distance-based decisions
* Highly sensitive to data distribution
* Behavior changes with small data perturbations

### Logistic Regression

* Global decision boundary
* Learns once, predicts consistently
* More robust and stable

👉 This is why Logistic Regression is preferred in production.

---

## 8️⃣ When to Use kNN

Use kNN when:

* Dataset is small
* Feature count is low
* Features are numeric
* You want a quick non-linear baseline
* You are doing exploratory analysis

---

## 9️⃣ When NOT to Use kNN

Avoid kNN when:

* Dataset is large
* Feature count is high
* Interpretability matters
* Prediction speed matters
* Model stability is critical

---

## 🔑 Interview-Ready One-Liners (Very Important)

* “kNN is distance-based, so feature scaling is mandatory.”
* “kNN suffers from the curse of dimensionality.”
* “kNN has no training phase but expensive inference.”
* “Logistic Regression is more stable due to regularization.”
* “kNN memorizes; Logistic Regression generalizes.”

---

## 🔟 Final Summary (Perfect Interview Answer)

> kNN is a simple, distance-based algorithm that can model non-linear patterns without assumptions, but it suffers from high variance, poor scalability, sensitivity to feature scaling, and the curse of dimensionality. In contrast, Linear and Logistic Regression are parametric, stable, interpretable models that generalize better and are more suitable for production systems.

---

